In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import os.path
from os import path

In [2]:
imdb = pd.read_csv('photo_metadata.csv')
wiki = pd.read_csv('wiki_photo_metadata.csv')
photo_df = pd.concat([imdb, wiki])

In [3]:
imdb2 = pd.read_csv('photo_metadata_second_faces.csv')
wiki2 = pd.read_csv('wiki_photo_metadata_second_faces.csv')
photo_df2 = pd.concat([imdb2, wiki2])

In [4]:
photo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 511817 entries, 0 to 59684
Data columns (total 8 columns):
name              511817 non-null object
dob               511817 non-null object
gender            511817 non-null float64
photo_taken       511817 non-null int64
age_when_taken    511817 non-null int64
file_path         511817 non-null object
face_location     511817 non-null object
face_score        511817 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 35.1+ MB


In [5]:
photo_df.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score
0,Fred Astaire,1899-05-10,1.0,1968,69,01/nm0000001_rm124825600_1899-5-10_1968.jpg,[1072.926 161.838 1214.784 303.696],1.459693
1,Fred Astaire,1899-05-10,1.0,1970,71,01/nm0000001_rm3343756032_1899-5-10_1970.jpg,[477.184 100.352 622.592 245.76 ],2.543198
2,Fred Astaire,1899-05-10,1.0,1968,69,01/nm0000001_rm577153792_1899-5-10_1968.jpg,[114.96964309 114.96964309 451.68657236 451.68...,3.455579
3,Fred Astaire,1899-05-10,1.0,1968,69,01/nm0000001_rm946909184_1899-5-10_1968.jpg,[622.88550564 424.21750384 844.33900767 645.67...,1.872117
4,Fred Astaire,1899-05-10,1.0,1968,69,01/nm0000001_rm980463616_1899-5-10_1968.jpg,[1013.85900236 233.88204221 1201.5861278 42...,1.158766


In [6]:
#A check to see if the file path exists
file_exists = 0
file_not_exist = 0

for i in photo_df.file_path:
    if path.exists('imdb_data/' + i):
        file_exists += 1
    elif path.exists('wiki_data/' + i):
        file_exists += 1
    else: 
        file_not_exist += 1
        
print(file_exists)
print(file_not_exist)

511817
0


In [7]:
photo_df.name.nunique()

72177

In [8]:
photo_df.count()

name              511817
dob               511817
gender            511817
photo_taken       511817
age_when_taken    511817
file_path         511817
face_location     511817
face_score        511817
dtype: int64

In [9]:
photo_df2 = photo_df2[photo_df2.second_face_score.isnull()]
photo_df2.drop(columns = {'second_face_score'}, axis = 1, inplace = True)
photo_df2.reset_index(drop=True)
photo_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298367 entries, 3 to 59684
Data columns (total 8 columns):
name              298367 non-null object
dob               298367 non-null object
gender            298367 non-null float64
photo_taken       298367 non-null int64
age_when_taken    298367 non-null int64
file_path         298367 non-null object
face_location     298367 non-null object
face_score        298367 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 20.5+ MB


In [11]:
photo_df2[photo_df2.name == 'Fred Astaire']

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score
3,Fred Astaire,1899-05-10,1.0,1968,69,01/nm0000001_rm946909184_1899-5-10_1968.jpg,[622.88550564 424.21750384 844.33900767 645.67...,1.872117
4,Fred Astaire,1899-05-10,1.0,1968,69,01/nm0000001_rm980463616_1899-5-10_1968.jpg,[1013.85900236 233.88204221 1201.5861278 42...,1.158766
57009,Fred Astaire,1899-05-10,1.0,1941,42,62/64962_1899-05-10_1941.jpg,[115.24513741 69.49908244 320.22238474 274.47...,4.236289
